In [1]:
!pip install -qU \
  langchain-core  \
  langchain[google-genai]  \
  langchain-community  \
  langsmith  \
  google-search-results 

## Tools

In [2]:
from langchain_core.tools import tool

@tool
def add(x: float, y:float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def subtract(x: float, y:float) -> float:
    """Subtract 'x' and 'y'."""
    return x - y

@tool
def multiply(x: float, y:float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def divide(x: float, y:float) -> float:
    """Divide 'x' and 'y'."""
    return x / y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y



In [3]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x7a893c569090>)

In [4]:
print(f"{add.name}\n{add.description}\n{add.args_schema.model_json_schema()}")

add
Add 'x' and 'y'.
{'description': "Add 'x' and 'y'.", 'properties': {'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}, 'required': ['x', 'y'], 'title': 'add', 'type': 'object'}


In [5]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

In [6]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"
llm_output_dict = json.loads(llm_output_string)
llm_output_dict

{'x': 5, 'y': 2}

In [7]:
exponentiate.func(**llm_output_dict)

25

In [8]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [9]:
import getpass
import os
from langchain_google_genai import GoogleGenerativeAI

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

#llm = GoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0) 


from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

Enter API key for Google Gemini: ········


In [10]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key = "chat_history",
    return_messages = True
)

/tmp/ipykernel_7353/2322671580.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [11]:
from langchain.agents import create_tool_calling_agent

tools = [add, subtract, multiply, divide, exponentiate]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

In [12]:
agent.invoke({
    "input": "What is 10.7 multiplied by 7.68",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps":[]
})


[ToolAgentAction(tool='multiply', tool_input={'y': 7.68, 'x': 10.7}, log="\nInvoking: `multiply` with `{'y': 7.68, 'x': 10.7}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"y": 7.68, "x": 10.7}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--4fd533f6-acf5-48f5-a27e-ebe77d06caba-0', tool_calls=[{'name': 'multiply', 'args': {'y': 7.68, 'x': 10.7}, 'id': 'e4f9ae0e-e940-4809-8250-1377bd269dd7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 268, 'output_tokens': 119, 'total_tokens': 387, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 95}})], tool_call_id='e4f9ae0e-e940-4809-8250-1377bd269dd7')]

In [13]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    memory = memory,
    verbose = True
)

In [14]:
agent_executor.invoke({
    "input": "What is 10.7 multiplied by 7.68",
    "chat_history": memory.chat_memory.messages
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'y': 7.68, 'x': 10.7}`


82.1759999999999910.7 multiplied by 7.68 is 82.176.

> Finished chain.


{'input': 'What is 10.7 multiplied by 7.68',
 'chat_history': [HumanMessage(content='What is 10.7 multiplied by 7.68', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={})],
 'output': '10.7 multiplied by 7.68 is 82.176.'}

In [15]:
memory.chat_memory.messages

[HumanMessage(content='What is 10.7 multiplied by 7.68', additional_kwargs={}, response_metadata={}),
 AIMessage(content='10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={})]

In [16]:
agent_executor.invoke({
    "input": "My name is Muhammad",
    "chat_history": memory.chat_memory.messages
})



> Entering new AgentExecutor chain...
Hello Muhammad, how can I help you?


> Finished chain.


{'input': 'My name is Muhammad',
 'chat_history': [HumanMessage(content='What is 10.7 multiplied by 7.68', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Muhammad', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Muhammad, how can I help you?\n', additional_kwargs={}, response_metadata={})],
 'output': 'Hello Muhammad, how can I help you?\n'}

In [17]:
agent_executor.invoke({
    "input": "What is nine plus 10, minus 4 * 2 to the power of 3",
    "chat_history": memory.chat_memory.messages
})



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'y': 3.0, 'x': 2.0}`


8.0
Invoking: `multiply` with `{'y': 8.0, 'x': 4.0}`


32.0
Invoking: `add` with `{'y': 10.0, 'x': 9.0}`


19.0
Invoking: `subtract` with `{'y': 32.0, 'x': 19.0}`


-13.0Nine plus 10, minus 4 * 2 to the power of 3 is -13.

> Finished chain.


{'input': 'What is nine plus 10, minus 4 * 2 to the power of 3',
 'chat_history': [HumanMessage(content='What is 10.7 multiplied by 7.68', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is 82.176.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Muhammad', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Muhammad, how can I help you?\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2 to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nine plus 10, minus 4 * 2 to the power of 3 is -13.', additional_kwargs={}, response_metadata={})],
 'output': 'Nine plus 10, minus 4 * 2 to the power of 3 is -13.'}

In [20]:
agent_executor.invoke({
    "input": "9+10-4*2**3",
    "chat_history": memory.chat_memory.messages
})

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 37
}
].




> Entering new AgentExecutor chain...


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 35
}
]

In [ ]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory.chat_memory.messages
})